In [71]:
import requests

from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import csv

In [ ]:
headers = requests.utils.default_headers()

headers.update(
    {
        'User-Agent': 'python-requests/2.31.0',
        'Accept-Encoding': 'gzip, deflate',
        'Accept': '*/*',
        'Connection': 'keep-alive',
    }
)

In [84]:
def samescrap(link, writer):
    # Send an HTTP GET request to the website
    response = requests.get(link, headers=headers)
    # Parse the HTML code using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    list_anime = soup.find_all(class_="dtla")
    for anime in list_anime:
        row = []
        entry = anime.findChildren('h2', recursive=False)[-1]
        link_ = entry.findChildren('a', recursive=False)[-1]['href']
        row.append(link_)
        headline_ = entry.findChildren('a', recursive=False)[-1]['title']
        row.append(headline_)

        added_info = anime.findChildren('span', recursive=True)
        episode_ = added_info[0].author.text
        row.append(episode_)
        posted_by_ = added_info[1].author.text
        row.append(posted_by_)

        inner_response = requests.get(link_, headers=headers)
        inner_soup = BeautifulSoup(inner_response.content, 'html.parser')

        if inner_soup.find(class_='nvs nvsc') is not None:
            main_link_ = inner_soup.find(class_='nvs nvsc').findChildren('a', recursive=False)[-1]['href']
        else:
            main_link_ = ''
        row.append(main_link_)
        
        if main_link_ == '':
            row.append('')
            row.append('')
            row.append('')
            row.append('')
            row.append('')
            row.append('')
            row.append('')
            row.append('')
            row.append('')
            row.append('')
            row.append('')
            row.append('')
            row.append('')
            row.append('')
            writer.writerow(row)
            break
        detail_response = requests.get(main_link_, headers=headers)
        detail_soup = BeautifulSoup(detail_response.content, 'html.parser')

        rate_ = detail_soup.find(class_='clearfix archiveanime-rating')
        if rate_ is not None:
            row.append(rate_.span.text)
        else:
            row.append('')
        
        ratingcount_ = detail_soup.find(class_='clearfix archiveanime-rating')
        if ratingcount_ is not None:
            row.append(ratingcount_.i.text)
        else:
            row.append('')

        genre_ = detail_soup.find(class_='genre-info')
        if genre_ is not None:
            genre_list_ = []
            for gen in genre_.findChildren('a', recursive=True):
                genre_list_.append(gen.text)
            row.append('|'.join(genre_list_))
        else:
            row.append('')

        specs_ = detail_soup.find(class_='spe').findChildren('span', recursive=True)
        japanese_title_ = ''
        english_title_ = ''
        synonims_ = ''
        status_ = ''
        type_ = ''
        source_ = ''
        duration_ = ''
        total_episode_ = ''
        season_ = ''
        studio_ = ''
        producers_ = ''
        released_ = ''
        if specs_ is not None:
            for elements in specs_:
                japanese_title_ = elements.text if elements.text.startswith('Japanese') else japanese_title_
                synonims_ = elements.text if elements.text.startswith('Synonyms') else synonims_
                english_title_ = elements.text if elements.text.startswith('English') else english_title_
                status_ = elements.text if elements.text.startswith('Status') else status_
                type_ = elements.text if elements.text.startswith('Type') else type_
                source_ = elements.text if elements.text.startswith('Source') else source_
                duration_ = elements.text if elements.text.startswith('Duration') else duration_
                total_episode_ = elements.text if elements.text.startswith('Total Episode') else total_episode_
                season_ = elements.text if elements.text.startswith('Season') else season_
                studio_ = elements.text if elements.text.startswith('Studio') else studio_
                producers_ = elements.text if elements.text.startswith('Producers') else producers_
                released_ = elements.text if elements.text.startswith('Released') else released_
        
        row.append(japanese_title_)
        row.append(synonims_)
        row.append(english_title_)
        row.append(status_)
        row.append(type_)
        row.append(source_)
        row.append(duration_)
        row.append(total_episode_)
        row.append(season_)
        row.append(studio_)
        row.append(producers_)
        row.append(released_)

        writer.writerow(row)

In [ ]:
links = []

# URL of the website to scrape
url = 'https://samehadaku.email/anime-terbaru/page/1/'
links.append(url)

# Send an HTTP GET request to the website
response = requests.get(url)

# Parse the HTML code using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')
n = soup.find_all(class_="arrow_pag")
next_page = n[-1].findChildren('i', recursive=False)[-1]['id']

while next_page == 'nextpagination':
    url = n[-1]['href']

    links.append(url)

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    n = soup.find_all(class_="arrow_pag")
    next_page = n[-1].findChildren('i', recursive=False)[-1]['id']
print(links)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [76]:
print(len(links))

462


In [83]:
main_page = ['link', 'headline', 'episode', 'posted_by']
inner_page = ['main_link']
detail_page = ['rate', 'ratingcount', 'genre', 'japanese_title', 'synonims', 'english_title', 'status', 'type', 'source', 'duration', 'total_episode', 'season', 'studio', 'producers', 'released']

with open('animelist.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    field = main_page + inner_page + detail_page
    
    writer.writerow(field)
    
    for link in links:
        samescrap(link, writer)

AttributeError: 'NoneType' object has no attribute 'findChildren'